In [1]:
import numpy as np
import matplotlib.pyplot as plt

from scipy.fftpack import dctn, fftn, fftshift, fftfreq
from scipy.io import loadmat
from scipy.interpolate import RectBivariateSpline, RectSphereBivariateSpline

from cora.signal import corr21cm
from cora.foreground import gaussianfg, galaxy

from util import *

In [2]:
path = "/Users/zhengzhang/PythonProjects/TIBEC/HIRAX_201.txt"
antenna_sph_coords = np.loadtxt(path,
                                comments=('// >>', '361 181', '#'),
                                usecols=(0, 1),
                                max_rows=361 * 181, ).reshape(-1, 2)


E1 = (np.loadtxt(path,
                      comments=('// >>', '361 181'),
                      usecols=(2, 4),
                      max_rows=361 * 181,
                      ) + 1j * np.loadtxt(path,
                                          comments=('// >>', '361 181'),
                                          usecols=(3, 5),
                                          max_rows=361 * 181,
                                          )).reshape(361 * 181, 2)


aux = antenna_sph_coords[:,1]
nphi, ntheta = 361, 89
E1 = E1[ np.where(np.logical_and(aux>0, aux<90)), :].reshape(nphi, ntheta, 2)
phis = np.deg2rad(np.arange(361))
thetas = np.deg2rad(np.arange(90)[1:])

/var/folders/69/cq4y0nj97qb_376qpg8lsjf80000gn/T/ipykernel_5337/837851083.py:2: UserWarning: Input line 1 contained no data and will not be counted towards `max_rows=65341`.  This differs from the behaviour in NumPy <=1.22 which counted lines rather than rows.  If desired, the previous behaviour can be achieved by using `itertools.islice`.
Please see the 1.23 release notes for an example on how to do this.  If you wish to ignore this warning, use `warnings.filterwarnings`.  This warning is expected to be removed in the future and is given only once per `loadtxt` call.
  antenna_sph_coords = np.loadtxt(path,
/var/folders/69/cq4y0nj97qb_376qpg8lsjf80000gn/T/ipykernel_5337/837851083.py:8: UserWarning: Input line 1 contained no data and will not be counted towards `max_rows=65341`.  This differs from the behaviour in NumPy <=1.22 which counted lines rather than rows.  If desired, the previous behaviour can be achieved by using `itertools.islice`.
Please see the 1.23 release notes for an ex

In [3]:
Pauli_I = 0.5 * np.array([[1., 0.],
                          [0., 1.]])
Pauli_Q = 0.5 * np.array([[1., 0.],
                          [0., -1.]])
Pauli_U = 0.5 * np.array([[0., 1.],
                          [1., 0.]])
Pauli_V = 0.5 * np.array([[0., -1.j],
                          [1.j, 0.]])

pauli_array = np.array([Pauli_I, Pauli_Q, Pauli_U, Pauli_V])    

In [4]:
Ndim=150
thetaMax = np.deg2rad(45)
radius = 2.*np.tan(thetaMax/2.)
x_coord, y_coord = np.linspace(-radius, radius, Ndim), np.linspace(-radius, radius, Ndim)
grid_y, grid_x = np.meshgrid(x_coord, y_coord)
target_phi, target_theta = plane2sphere_v2(grid_x.flatten(), grid_y.flatten()) # In degrees

B_matrix = np.einsum("fsi, pim, fsm -> sfp",
                             np.conjugate(E1), # Field in Cartesian grid.
                             pauli_array,
                             E1).real

# Interpolation:
B_matrix = interpolation(Ndim, phis, thetas, B_matrix, target_phi, target_theta)

# Rescaling power density for the projected field
grid_target_theta = target_theta.reshape(grid_x.shape)
B_matrix = Beam_scaled(B_matrix, grid_target_theta)
# Normalize the beam
normalization_factor = 1 / np.sum(B_matrix[:, :, 0])
B_matrix *= normalization_factor

#B_matrix = directional_window(B_matrix, grid_target_theta, 50.)



del E1, antenna_sph_coords, grid_target_theta, grid_x, grid_y, target_phi, target_theta


In [5]:
print("X_k: min={}, max={}".format(1/np.deg2rad(10), 1/np.deg2rad(1)))

X_k: min=5.729577951308232, max=57.29577951308232


In [6]:
# Beam pixel resolutions
x_res = np.abs(x_coord[1] - x_coord[0])
y_res = np.abs(y_coord[1] - y_coord[0])

x_fft_coords = fftshift(fftfreq(x_coord.size, d=x_res))
y_fft_coords = fftshift(fftfreq(y_coord.size, d=y_res))
                                
Beam_fft = fftshift(fftn(B_matrix, axes=(0, 1)), axes=(0, 1))

del B_matrix
# plt.imshow(np.abs(Beam_fft[:, :, 0])) 

In [7]:
"""
bln = 15. # In units of wavelength.
d, (q_x, q_y) =  q_matrix(x_fft_coords, y_fft_coords, bln*np.sqrt(3)) # q_array: [q_x[...], q_y[...]]
total_beam_matrix = np.array([4*FFT_Beam_matrix(Beam_fft, d, 0, 0),
                              FFT_Beam_matrix(Beam_fft, d, bln, 0), 
                             2*FFT_Beam_matrix(Beam_fft, d, bln, 60),
                             2*FFT_Beam_matrix(Beam_fft, d, bln, 120),
                              FFT_Beam_matrix(Beam_fft, d, bln*np.sqrt(3), 90) ])[:, :, :, :3]

"""

'\nbln = 15. # In units of wavelength.\nd, (q_x, q_y) =  q_matrix(x_fft_coords, y_fft_coords, bln*np.sqrt(3)) # q_array: [q_x[...], q_y[...]]\ntotal_beam_matrix = np.array([4*FFT_Beam_matrix(Beam_fft, d, 0, 0),\n                              FFT_Beam_matrix(Beam_fft, d, bln, 0), \n                             2*FFT_Beam_matrix(Beam_fft, d, bln, 60),\n                             2*FFT_Beam_matrix(Beam_fft, d, bln, 120),\n                              FFT_Beam_matrix(Beam_fft, d, bln*np.sqrt(3), 90) ])[:, :, :, :3]\n\n'

In [8]:
bln = 10. # In units of wavelength.
d, (q_x, q_y) =  q_matrix(x_fft_coords, y_fft_coords, np.sqrt(3)*bln) # q_array: [q_x[...], q_y[...]]
total_beam_matrix = np.array([FFT_Beam_matrix(Beam_fft, x_fft_coords, y_fft_coords, d, bln, 0),
                              2*FFT_Beam_matrix(Beam_fft, x_fft_coords, y_fft_coords, d, bln, 60),
                              2*FFT_Beam_matrix(Beam_fft, x_fft_coords, y_fft_coords, d, bln, 120),
                              FFT_Beam_matrix(Beam_fft, x_fft_coords, y_fft_coords, d, np.sqrt(3)*bln, 90)], 
                             dtype=np.complex64 )[:, :, :, :3]
q_absolute = 2 * np.pi * np.sqrt(q_x**2 + q_y**2)

In [ ]:
dim = q_x.shape[0]
mixing_matrix = np.zeros(shape=(dim,dim,dim,dim))
sigma = np.deg2rad(1)
for y in range(dim):
    for x in range(dim): 
        aux = np.exp( ((q_x - q_x[y,x])**2 + (q_y - q_y[y,x])**2 )* sigma**2 / 2.)
        mixing_matrix[y,x, :, :] = aux

In [ ]:
"""
beam_err_matrix = np.zeros(shape=(5,dim,dim,dim,dim,3), dtype=complex)

for y in range(dim):
    for x in range(dim): 
        aux = np.exp( ((q_x - q_x[y,x])**2 + (q_y - q_y[y,x])**2 )* sigma**2 / 2.)
        beam_err_matrix[:, y, x, :, :, :] = aux[np.newaxis, :, :, np.newaxis] * total_beam_matrix
"""

In [ ]:
n_bln = 4
mixing_matrix = mixing_matrix.reshape(dim*dim, dim*dim)
total_beam_matrix = total_beam_matrix.reshape(n_bln, dim*dim, 3)

In [ ]:
delta_beam_matrix = np.zeros(shape=(n_bln,dim*dim,dim*dim,3), dtype=complex)
for bl in range(delta_beam_matrix.shape[0]):
    for pol in range(delta_beam_matrix.shape[-1]):
        delta_beam_matrix[bl, :, :, pol] = mixing_matrix @ np.diag(total_beam_matrix[bl, :, pol])
        
del total_beam_matrix

In [ ]:
del mixing_matrix

delta_beam_i = delta_beam_matrix[:, :, :, 0].reshape(n_bln*dim*dim,dim*dim) 

delta_beam_p = delta_beam_matrix[:, :, :, 1:].reshape(n_bln*dim*dim,dim*dim*2) 

del delta_beam_matrix


In [ ]:
BBt_inv = delta_beam_p.conj().T @ delta_beam_p # (2Ns, Nd) x (Nd, 2Ns)

In [ ]:
BBt_inv = np.linalg.inv(BBt_inv) # (2Ns, 2Ns)

In [ ]:
AtB = delta_beam_i.conj().T @ delta_beam_p # (Ns, Nd) x (Nd, 2Ns)

In [ ]:
AtB_BBt_inv = AtB @ BBt_inv  # (Ns, 2Ns) x (2Ns, 2Ns)
del BBt_inv

In [ ]:
AtB_BBt_inv@AtB.conj().T

In [ ]:
aux1 = aux1 @ delta_beam_p.conj().T   # (Ns, 2Ns) x (2Ns, Nd)
U = delta_beam_i.conj().T - aux1 # (Ns,  Nd)
del aux1

In [ ]:
K = U @ delta_beam_i # (Ns,  Nd) x (Nd, Ns)

In [ ]:
K = np.linalg.inv(K)

In [ ]:
K = K @ U
del U

In [ ]:
delta_beam_i = K@delta_beam_i

In [ ]:
delta_beam_p = K@delta_beam_p
del K

In [ ]:
def MLE_oper_marginalize_pol_wo_prior(matr_A, matr_B):
    print("Step 1 ...")
    aux1 = matr_B.conj().T @ matr_B
    print("Step 2 ...")
    aux1 = np.linalg.inv(aux1)
    print("Step 3 ...")
    aux2 = matr_A.conj().T @ matr_B
    aux1 = aux2 @ aux1
    del aux2 
    aux1 = aux1 @ matr_B.conj().T
    aux1 = matr_A.conj().T - aux1 
    C = aux1@matr_A
    C = np.linalg.inv(C)
    C = C @ aux1
    return C

In [ ]:
K = MLE_oper_marginalize_pol_wo_prior(delta_beam_i, delta_beam_p)

In [ ]:
delta_beam_i = K@delta_beam_i
delta_beam_p = K@delta_beam_p
del K

In [9]:
freq = 400.
pol_frac = 0.6

fsyn = galaxy.FullSkySynchrotron()
fpol = galaxy.FullSkyPolarisedSynchrotron()
vectorized_syn_aps = np.vectorize(fsyn.angular_powerspectrum)
vectorized_pol_aps = np.vectorize(fpol.angular_powerspectrum)

q_absolute=q_absolute.flatten()
q_abs_filtered = q_absolute[np.where(q_absolute>5)]

cv_fg_i = vectorized_syn_aps(q_abs_filtered, freq, freq)
cv_fg_p = np.zeros(shape=np.append(q_abs_filtered.shape, 2))
cv_fg_p[:, 0] = pol_frac * vectorized_pol_aps(q_abs_filtered, freq, freq)
cv_fg_p[:, 1] = pol_frac * vectorized_pol_aps(q_abs_filtered, freq, freq)

np.savetxt('cv_fg_i.out', cv_fg_i.reshape(-1), delimiter=',') 
np.savetxt('cv_fg_p.out', cv_fg_p.reshape(-1), delimiter=',') 

In [ ]:
P_delta_I = aux@cv_fg_i@aux.conj().T + aux@cv_fg_i + cv_fg_i@aux.conj().T
P_delta_p = aux@cv_fg_p@aux.conj().T 

In [ ]:
q_absolute=q_absolute.flatten()

In [ ]:
q_absolute[np.where(q_absolute[:4]>5)].shape

In [ ]:
np.arange(36).reshape(6,6)[:2,np.where(q_absolute[:4]>5)]

In [ ]:
87*87

In [ ]:
np.where(q_absolute<3) 

In [ ]:
vectorized_pol_aps(aux, freq, freq)

In [ ]:
np.array((np.arange(5), np.arange(5)+1)).swapaxes(1, 0)

In [ ]:
cv_fg_i